In [35]:
# Torch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

# Numpy Imports
import numpy as np

# Matplotlib Imports
from matplotlib import pyplot as plt

# Feature Engineering Import
%run "../Feature Engineering/azwad.ipynb"

In [34]:
# Setting up the Tensors for the model
data = featureEngineering("../Datasets/spam.csv")
x_train = torch.Tensor(torch.from_numpy(data[0].values.astype(np.float32)))
x_test = torch.Tensor(torch.from_numpy(data[1].values.astype(np.float32)))
y_train = torch.Tensor(torch.from_numpy(data[2].values.astype(np.float32))).reshape(-1,1)
y_test = torch.Tensor(torch.from_numpy(data[3].values.astype(np.float32))).reshape(-1,1)

In [48]:
class SimpleNeuralNet(nn.Module):
    def __init__(self, hidden_units, num_dimensions, num_classes):
        super().__init__()
        self.num_dimensions = num_dimensions
        self.linear = nn.Linear(num_dimensions, hidden_units)
        self.output_layer = nn.Linear(hidden_units, num_classes)

    def forward(self, x):
        x = x.view(-1, self.num_dimensions)
        z1 = self.linear(x)
        a1 = F.relu(z1)
        z2 = self.output_layer(a1)
        return F.log_softmax(z2, dim=1)

nn_model = SimpleNeuralNet(20, 3900, 2)
#summary(nn_model, (1, 28, 28))

ValueError: Using a target size (torch.Size([3900, 1])) that is different to the input size (torch.Size([1, 2])) is deprecated. Please ensure they have the same size.